In [37]:
prices_path = "prices.txt"
products_path = "products.txt"
sales_path = "sales2.txt"

In [2]:
from pyspark import SparkConf, SparkContext

In [40]:

def modify_sales_date(row):
    date = row.split(",")[1]
    year = date.split("/")[0]

    new_row = row.split(",")[0] + "," + year + "," + row.split(",")[2]
    return new_row

def make_key_value_pairs(row):
    return((row.split(",")[0], row.split(",")[1]), row.split(",")[2])

def modify_key_value_pair(row):
    return (row[0][0], (row[0][1], row[1]))

def filter_condition(row):
    values = row[1]
    if len(values) > 1:
        pair1 = values[0]
        year1 = pair1[0]
        total1 = pair1[1]

        pair2 = values[1]
        year2 = pair2[0]
        total2 = pair2[1]

        if year1 == '2021':
            if total1 < total2:
                return True
        else:
            if total2 < total1:
                return True
        return False
    else:
        pair1 = values[0]
        year1 = pair1[0]
        if year1 == '2019':
            return True
    return False

def keep_max_prods(row):
    values = row[1]
    max_val = 0
    for v in values:
        if v[1] > max_val:
            max_val = v[1]
    new_values = [v for v in values if v[1] == max_val]
    new_row = [(row[0], v[0]) for v in new_values]    
    return new_row

if __name__ == "__main__":
    conf = SparkConf().setAppName("lab 7")
    ctx = SparkContext.getOrCreate(conf)

    # Ex1
    sales_RDD = ctx.textFile(sales_path)
    sales_RDD = sales_RDD.map(modify_sales_date)
    # KEEP only 2019 and 2021
    filtered_sales_RDD = sales_RDD.filter(lambda row: row.split(",")[1] == '2019' or row.split(",")[1] == '2021')
    filtered_sales_RDD = filtered_sales_RDD.map(make_key_value_pairs)

    reduced_sales = filtered_sales_RDD.reduceByKey(lambda v1, v2: int(v1) + int(v2))
    reduced_sales = reduced_sales.map(modify_key_value_pair)
    # reduced_sales = reduced_sales.reduceByKey()
    reduced_sales = reduced_sales.groupByKey().mapValues(lambda values: list(values))
    filtered_RDD = reduced_sales.filter(filter_condition)

    final_RDD_ex1 = filtered_RDD.map(lambda x: x[0]) 
    final_RDD_ex1.saveAsTextFile("ex1")

    # EX2
    sales_RDD = ctx.textFile(sales_path)
    # Make key-value pairs
    sales_RDD = sales_RDD.map(lambda row: ((row.split(",")[0], row.split(",")[1].split("/")[0]), 1))
    reduced_RDD = sales_RDD.reduceByKey(lambda v1, v2: v1+v2)

    rdd_year_key = reduced_RDD.map(lambda x: (x[0][1], (x[0][0], x[1])))
    print(rdd_year_key.take(10))
    grouped_rdd = rdd_year_key.groupByKey().mapValues(lambda v: list(v))
    max_prod_for_each_year = grouped_rdd.flatMap(keep_max_prods)

    max_prod_for_each_year.saveAsTextFile("ex2")
    print(max_prod_for_each_year.take(10))

[('2021', ('pid2', 4)), ('2021', ('pid3', 4)), ('2020', ('pid2', 4)), ('2020', ('pid3', 4)), ('2019', ('pid2', 4)), ('2019', ('pid3', 4)), ('2018', ('pid2', 4)), ('2018', ('pid3', 4)), ('2021', ('pid1', 4)), ('2020', ('pid1', 4))]
[('2021', 'pid2'), ('2021', 'pid3'), ('2021', 'pid1'), ('2020', 'pid2'), ('2020', 'pid3'), ('2020', 'pid1'), ('2019', 'pid2'), ('2019', 'pid3'), ('2019', 'pid1'), ('2018', 'pid2')]
